In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import itertools
from setuptools.namespaces import flatten
from scipy import stats
import GetOldTweets3 as got


Bad key "text.kerning_factor" on line 4 in
C:\Users\User\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# function created to form the dataset
def get_tweets(tag, state, startdate, enddate, maxtweet):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(tag)\
                                            .setSince(startdate)\
                                            .setUntil(enddate)\
                                            .setNear(state)\
                                            .setWithin("400mi")\
                                            .setMaxTweets(maxtweet)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    text_tweets = [[tw.username,
                tw.text,
                tw.date,
                tw.retweets,
                tw.favorites,
                tw.mentions,
                tw.hashtags,
                tw.geo] for tw in tweet]
    df_state= pd.DataFrame(text_tweets, columns = ['User', 'Text', 'Date', 'Favorites', 'Retweets', 'Mentions','Hashtags', 'Geolocation'])
    
    return df_state

In [10]:
df_final.head()

,User,Text,Date,Favorites,Retweets,Mentions,Hashtags,Geolocation
0,kojy1402,Q haga como dijo el Ab de #DanielSalcedo en cu...,2020-07-14 22:38:13+00:00,0,0,,#DanielSalcedo #corrupcionecuador,
1,GamesRedDeer,PLAY & Let The Fun Begin Like + RT + Click htt...,2020-07-28 20:47:23+00:00,138,350,,,
2,Jornarmo,SOLAMENTE PREGUNTEN A ESTE ABOGADITO ¿PORQUÉ D...,2020-07-14 21:26:39+00:00,0,1,,,
3,aljomec,"Reviso tendencias y veo una tal ""Emma Guerrero...",2020-07-14 20:47:35+00:00,18,55,,,
4,mashiedu09,"Que asco, después de haber entregado los hospi...",2020-07-14 17:30:26+00:00,0,0,,,


In [4]:
#pd.Series(np.array(''.join(list(abc.Hashtags[abc.Hashtags!= '']) ).replace(' ','').split('#'))[ 
#np.array(''.join(list(abc.Hashtags[abc.Hashtags!= '']) ).replace(' ','').split('#'))!='']).value_counts().head(15)

In [5]:
# keywords related to IESS corruption in ecuador
lista_df= []
for clave in ['daniel salcedo', 'Abraham Muñoz', 'iess', 'carnet discapacitado','Reparto Hospitales',
             '#ElPeorGobiernoDeLaHistoria','Bucaram', 'corrupcion ecuador', '#EcuadorEnCrisis',"daniel mendoza"]:
    df_tag= get_tweets(tag=clave ,state='-1.83, -78.18', startdate="2020-04-01", enddate="2020-07-15",maxtweet= 5000)
    lista_df.append(df_tag)
    
    
    

In [8]:
df_final= pd.concat(lista_df)

In [12]:
# tweets with mentioned people
df_solo_ment= df_final[df_final.Mentions!= ''].reset_index()
df_solo_ment.User= df_solo_ment.User.str.lower()
df_solo_ment.Mentions= df_solo_ment.Mentions.str.lower()


In [13]:
# dictionary data processing for gephi:
user_unico= list(set(df_solo_ment.User))
dic_menc= dict()
for usuario in user_unico:
    lista_quienes_mencione= np.array(''.join( list( df_solo_ment.loc[df_solo_ment.User== usuario,'Mentions'] ) ).replace(' ','').split('@'))[
    np.array(''.join( list( df_solo_ment.loc[df_solo_ment.User== usuario,'Mentions'] ) ).replace(' ','').split('@'))!=''].tolist()
    mencione_df= pd.DataFrame(pd.Series(lista_quienes_mencione).value_counts()).reset_index()
    mencione_df= mencione_df.to_records(index=False)
    lista_tupla= list(mencione_df)
    dic_menc[usuario]= lista_tupla
    

In [14]:
# data processing for gephi: origin, arrival(mentioned people), weight( times mentioned)
listaOrigen=[]
listaLlegada=[]
listaArcos=[]
for idUser in dic_menc:
    listaOrigen.extend([idUser]*len(dic_menc[idUser]))
    listaLlegada.extend( list( dict(dic_menc[idUser]).keys()) )
    listaArcos.extend( list( dict(dic_menc[idUser]).values()) )
    

In [31]:
(set(set(listaOrigen) | set(listaLlegada)))

2179

In [15]:
# data processing for gephi
data = pd.DataFrame(np.array(listaOrigen))
data.rename(columns={0:'Source'},inplace=True)
data['Target']= np.array(listaLlegada)
data['Weight']= np.array(listaArcos)
data['Label']= np.array(['conexion']*len(listaArcos))
data['Type']= np.array(['directed']*len(listaArcos))

In [34]:
# df for users
nod = pd.DataFrame(np.array( list( (set(set(listaOrigen) | set(listaLlegada))))))
nod.rename(columns={0:'ID'},inplace=True)
nod['Label']= nod['ID']
#nod.to_excel('nod.xlsx')

In [19]:
# export dataset for gephi
#data.to_excel('sna_IESS.xlsx')

In [20]:
# export full data from twitter
#df_final.to_csv('full_data.csv')

In [70]:
lista_df2= []
for clave2 in ['Bucaram', 'Abdala Bucaram','Jacobo Bucaram', 'Michel Bucaram' ]:
    df_tag2= get_tweets(tag=clave ,state='-1.83, -78.18', startdate="2020-08-08", enddate="2020-08-10",maxtweet= 5000)
    lista_df2.append(df_tag2)

In [71]:
df_final2= pd.concat(lista_df)

In [74]:
#df_final2[df_final2.Mentions!='']

In [75]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import re


In [ ]:
# Access token :118477145-taYbmK7AkERnNMQp8ydZKjIoMeDd9BMAxIhtAcQC
# Access token secret :CQKJcprX34wggv3QpvUwOBxNNmm27XpBhmuGuFAYHTi3y
# API key: ksgb4RP3K54oJS57v9dnIxLk2
# API secret key: wnlsli8KqXpmi2Sy4aRAdtacdhIT5vnqMXvgJbOzL5sMC9oPBx

# Enter Twitter API Keys
access_token = "118477145-taYbmK7AkERnNMQp8ydZKjIoMeDd9BMAxIhtAcQC"
access_token_secret = "CQKJcprX34wggv3QpvUwOBxNNmm27XpBhmuGuFAYHTi3y"
consumer_key = "ksgb4RP3K54oJS57v9dnIxLk2"
consumer_secret = "wnlsli8KqXpmi2Sy4aRAdtacdhIT5vnqMXvgJbOzL5sMC9oPBx"

# Create tracklist with the words that will be searched for
tracklist = ['Bucaram', 'Abdala Bucaram','Jacobo Bucaram', 'Michel Bucaram']
# Initialize Global variable
tweet_count = 0
# Input number of tweets to be downloaded
n_tweets = 30000

# Create the class that will handle the tweet stream
class StdOutListener(StreamListener):
      
    def on_data(self, data):
        global tweet_count
        global n_tweets
        global stream
        if tweet_count < n_tweets:
            print(data)
            tweet_count += 1
            return True
        else:
            stream.disconnect()

    def on_error(self, status):
        print(status)



# Handles Twitter authetification and the connection to Twitter Streaming API
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)
stream.filter(track=tracklist)

{"created_at":"Sun Aug 09 23:07:26 +0000 2020","id":1292598391957008384,"id_str":"1292598391957008384","text":"RT @radiolacalle: #RayonesPol\u00edticos El audio demostrar\u00eda una supuesto manejo de la justicia por parte de Bucaram, el gobierno no ha emitido\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":421536358,"id_str":"421536358","name":"Alfonso P\u00e1rraga","screen_name":"Cantante777","location":"Ecuador","url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":135,"friends_count":24,"listed_count":0,"favourites_count":65420,"statuses_count":61672,"created_at":"Sat Nov 26 02:29:47 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":null,"contr

{"created_at":"Sun Aug 09 23:07:28 +0000 2020","id":1292598398361706499,"id_str":"1292598398361706499","text":"RT @rodolfoasar: \ud83d\udc47\ud83c\udffd https:\/\/t.co\/k9vcaqD2wE","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":186677510,"id_str":"186677510","name":"Carlis \ud83d\udc9c\ud83d\udc9a","screen_name":"CarFY22","location":null,"url":null,"description":"Una chica muy alegre, apasionada y terca!!!\n\nContigo, sin ti.","translator_type":"none","protected":false,"verified":false,"followers_count":248,"friends_count":215,"listed_count":7,"favourites_count":17574,"statuses_count":30757,"created_at":"Sat Sep 04 02:44:39 +0000 2010","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":null,"contributors_enabled":false,"

{"created_at":"Sun Aug 09 23:07:28 +0000 2020","id":1292598400567967747,"id_str":"1292598400567967747","text":"RT @socialholico: Maldito sea aquel que permiti\u00f3 que Abdal\u00e1 Bucaram regrese al pa\u00eds, mil veces maldito.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":239156730,"id_str":"239156730","name":"Keka Galindo.\ud83d\udc83\ud83c\udffb\ud83d\udc8d","screen_name":"pocholate__","location":"El huevo","url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":993,"friends_count":199,"listed_count":0,"favourites_count":9319,"statuses_count":12318,"created_at":"Sun Jan 16 23:41:14 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":null,"contributors_

{"created_at":"Sun Aug 09 23:07:36 +0000 2020","id":1292598434348892163,"id_str":"1292598434348892163","text":"RT @luisevivanco: 1) Por qu\u00e9 el fulano pide garant\u00edas de seguridad a Bucaram? \n2) Por qu\u00e9 Bucaram habla de que a ese juez no lo puede contr\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":335680182,"id_str":"335680182","name":"Silvia","screen_name":"silviajaeda","location":null,"url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":22,"friends_count":278,"listed_count":0,"favourites_count":1040,"statuses_count":1907,"created_at":"Fri Jul 15 02:13:17 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":null,"contributors_enabled"

{"created_at":"Sun Aug 09 23:07:39 +0000 2020","id":1292598446008958979,"id_str":"1292598446008958979","text":"RT @AlmaMiaEcu: El Ecuador de @lenin Moreno, Nebot, Bucaram, Lasso y los lentejeros AP, no se convirti\u00f3 en Venezuela sino en Colombia del t\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1230377565333880832,"id_str":"1230377565333880832","name":"Franc Mendez","screen_name":"FrancMendez3","location":null,"url":null,"description":"soy un hombre serio y sincero, no le hago da\u00f1o a nadie, pero alguien no quiere verme aqu\u00ed,, es la tercera cuenta que habro y espero q no me bloqueen otra vez...","translator_type":"none","protected":false,"verified":false,"followers_count":184,"friends_count":342,"listed_coun

...

{"created_at":"Sun Aug 09 23:17:11 +0000 2020","id":1292600845041885184,"id_str":"1292600845041885184","text":"RT @lahistoriaec: Al igual que su padre, Jacobo Bucaram tambi\u00e9n habl\u00f3 por tel\u00e9fono con Shy Dahan, quien lo involucr\u00f3 en proceso por venta d\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":305743790,"id_str":"305743790","name":"Wladimir","screen_name":"WladimirWlaWla","location":null,"url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":150,"friends_count":456,"listed_count":0,"favourites_count":22550,"statuses_count":17013,"created_at":"Thu May 26 18:42:12 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":null,"contributor

{"created_at":"Sun Aug 09 23:17:12 +0000 2020","id":1292600850804695040,"id_str":"1292600850804695040","text":"RT @ErmelRosario: \ud83d\udc7aGUERRA ENTRE BANDAS PIRATA Y COMEGUATITA\ud83d\udc79\ud83d\udc79\n#Hampones\n\u00bfCu\u00e1l de las dos bandas lo mat\u00f3?\n La banda del gobierno captura y p\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1088208681852841984,"id_str":"1088208681852841984","name":"RoCa #LMLPQTP \ud83c\udfb5\ud83c\udfb5","screen_name":"ro_ca_g","location":null,"url":null,"description":"\u201cLos pobres y clase media que votan por la derecha, son como los perros... cuidan la mansi\u00f3n, pero duermen afuera\u201d","translator_type":"none","protected":false,"verified":false,"followers_count":325,"friends

{"created_at":"Sun Aug 09 23:17:14 +0000 2020","id":1292600855875801094,"id_str":"1292600855875801094","text":"RT @RamiroRiveraEc: Abdala Bucaram act\u00faa como un presunto mafioso del bajo mundo. Alardea de influencias y amistades entre matones y ladron\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":439197639,"id_str":"439197639","name":"Gabriel Menc\u00edas","screen_name":"gabrielmencias","location":null,"url":null,"description":"Descomplicado, alegre, divertido, luchador, me encanta hacer amigos y disfrutar de las cosas sencillas de la vida!","translator_type":"none","protected":false,"verified":false,"followers_count":87,"friends_count":209,"listed_count":0,"favourites_count":1889,"statuses_count":1741,"created_at":"Sa

{"created_at":"Sun Aug 09 23:17:15 +0000 2020","id":1292600860183339008,"id_str":"1292600860183339008","text":"RT @ramiroaguilart: Recapitulemos: asesinan a colaborador de la @FiscaliaEcuador en la c\u00e1rcel cuando era pieza clave en contra de los #Buca\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1108586375807410176,"id_str":"1108586375807410176","name":"\u062f\u0627\u0646\u064a\u0627\u0644","screen_name":"__Daniel_EC","location":null,"url":null,"description":"Cada d\u00eda mejor que ayer. Mamado de ver c\u00f3mo in\u00fatiles administran mi pa\u00eds. Es tiempo de cambiar de ra\u00edz","translator_type":"none","protected":false,"verified":false,"followers_count":25,"friends_count":151,"listed_count":0,"favourites_count":8375,"status

{"created_at":"Sun Aug 09 23:17:18 +0000 2020","id":1292600875924566017,"id_str":"1292600875924566017","text":"@UnTatuador Bucaram tiene sus aliados tambi\u00e9n https:\/\/t.co\/zZ47n9JAHK","display_text_range":[12,45],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":1292556070368481280,"in_reply_to_status_id_str":"1292556070368481280","in_reply_to_user_id":352613078,"in_reply_to_user_id_str":"352613078","in_reply_to_screen_name":"UnTatuador","user":{"id":1095878740486295553,"id_str":"1095878740486295553","name":"\u00ae","screen_name":"RONNYH2D","location":null,"url":null,"description":"Seg\u00f3 los pasos que manejan mi destino \ud83d\udd03\nF\u00daTBO \u26bd\nBarcelonistas de \ud83d\udc9b \u00eddolo del Ecuador @barcelonaSC \ud83c\uddea\ud83c\udde8\n@fcBarcelona \ud83c\uddea\ud83c\uddf8\n@FCBayern \ud83c\udde9\ud83c\uddea","translator_type":"none","protected":false,"v

In [ ]:
import json

In [ ]:
tweets_data_path = "twitter_data.txt"  
tweets_data = []  
tweets_file = open(tweets_data_path, "r")  
for line in tweets_file:  
    try:  
        tweet = json.loads(line)  
        tweets_data.append(tweet)  
    except:  
        continue
tweets = pd.DataFrame()
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['Username'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['Timestamp'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['Rt'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['Fv'] = list(map(lambda tweet: tweet['favourites_count'], tweets_data))
tweets.head()